# definitions_testscript.ipynb

A script for testing and development of automatic generation of `ArchetypeBuildingModel.jl` definitions from AmBIENCe data.

In [1]:
import ambience2abm as amb

### Read the raw data and assumptions.

ambience = amb.AmBIENCeDataset(
    interior_node_depth=0.1,
    period_of_variations=1209600
)

C:\AmBIENCe2ABM\src\ambience2abm\process_ambience_data.py:146: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  agg_data = data.groupby(cols).agg(


## First, let's quickly check the building fabrics and building nodes.

These are assumptions corresponding to the `ZonWallInt_B` RC-model in the AmBIENCe data
as closely as possible to define for `ArchetypeBuildingModel.jl`.

In [2]:
### Test definitions processing and check building fabrics

defs = amb.ABMDefinitions(
    ambience,
    aggregate_building_type=False, # First no aggregation
    aggregate_building_period=False
)
defs.data

,REFERENCE BUILDING CODE,building_stock,location_id,building_type,building_period,number_of_buildings,average_gross_floor_area_m2_per_building,category,number_of_storeys,reference_floor_area_m2,...,period_high,min_period_year,max_period_year,scope_types,scope_period_start_year,scope_period_end_year,building_scope,total_gross_floor_area_m2,total_gross_floor_area_m2_per_scope,weight_within_scope
0,AT-OFF-1850-1944-00,AmBIENCe_2016_AT_nonres,AT,OFF,1850-1944,7348.433395,688.582141,nonres,6,114.763690,...,1944,1850,2021,OFF,1850,1944,AT_OFF_1850_1944,5.060000e+06,5.060000e+06,1.0
1,AT-OFF-1945-1969-00,AmBIENCe_2016_AT_nonres,AT,OFF,1945-1969,9851.953631,747.059951,nonres,6,124.509992,...,1969,1850,2021,OFF,1945,1969,AT_OFF_1945_1969,7.360000e+06,7.360000e+06,1.0
2,AT-OFF-1970-1979-00,AmBIENCe_2016_AT_nonres,AT,OFF,1970-1979,4664.597308,765.339378,nonres,6,127.556563,...,1979,1850,2021,OFF,1970,1979,AT_OFF_1970_1979,3.570000e+06,3.570000e+06,1.0
3,AT-OFF-1980-1989-00,AmBIENCe_2016_AT_nonres,AT,OFF,1980-1989,2727.357440,843.307139,nonres,6,140.551190,...,1989,1850,2021,OFF,1980,1989,AT_OFF_1980_1989,2.300000e+06,2.300000e+06,1.0
4,AT-OFF-1990-1999-00,AmBIENCe_2016_AT_nonres,AT,OFF,1990-1999,300.000000,900.000000,nonres,6,150.000000,...,1999,1850,2021,OFF,1990,1999,AT_OFF_1990_1999,2.700000e+05,2.700000e+05,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2240,SK-ABL-1921-1945-00,AmBIENCe_2016_SK_res,SK,ABL,1921-1945,4191.134302,967.300000,res,5,256.000000,...,1945,1850,2021,ABL,1921,1945,SK_ABL_1921_1945,4.054084e+06,4.054084e+06,1.0
2241,SK-ABL-1946-1960-00,AmBIENCe_2016_SK_res,SK,ABL,1946-1960,12653.368056,744.940000,res,3,315.900000,...,1960,1850,2021,ABL,1946,1960,SK_ABL_1946_1960,9.426000e+06,9.426000e+06,1.0
2242,SK-ABL-1961-1980-00,AmBIENCe_2016_SK_res,SK,ABL,1961-1980,4809.491979,2812.480000,res,8,453.600000,...,1980,1850,2021,ABL,1961,1980,SK_ABL_1961_1980,1.352660e+07,1.352660e+07,1.0
2243,SK-ABL-1981-1994-00,AmBIENCe_2016_SK_res,SK,ABL,1981-1994,1562.612422,6949.260000,res,9,972.000000,...,1994,1850,2021,ABL,1981,1994,SK_ABL_1981_1994,1.085900e+07,1.085900e+07,1.0


In [3]:
### Check given parameters

[
    defs.room_height_m,
    defs.weather_start,
    defs.weather_end,
    defs.partition_wall_length_ratio_to_external_walls_m_m,
    defs.window_area_thermal_bridge_surcharge_W_m2K
]

[2.6, '2016-01-01', '2016-01-02', 0.5, 0.0]

In [4]:
### Check building fabrics

defs.building_fabrics

,building_fabrics,interior_air_and_furniture_weight
building_node,,
indoor_air,ZonWallIntB,1
internal_mass,ZonWallIntB,0
envelope_mass,ZonWallIntB,0


In [5]:
### Check building nodes and structure types

defs.building_node__structure_type

,building_node
structure_type,
partition_wall,internal_mass
light_partition_wall,internal_mass
separating_floor,internal_mass
roof,envelope_mass
base_floor,envelope_mass
exterior_wall,envelope_mass
light_exterior_wall,envelope_mass


Seems ok.

## Next, let's look at the archetype building definitions.

Each row of the raw AmBIENCe data corresponds to an archetype building model in
`ArchetypeBuildingModel.jl`, so let's examine the properties a bit.

In [6]:
### Check building archetype params

archetype_data = defs.building_archetype()
archetype_data

c:\Users\trtopi\AppData\Local\miniconda3\envs\atlitetoolbox\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,building_scope,building_fabrics,building_frame_depth_m,number_of_storeys,weather_start,weather_end,room_height_m,window_area_to_external_wall_ratio_m2_m2,partition_wall_length_ratio_to_external_walls_m_m,window_area_thermal_bridge_surcharge_W_m2K,reference_floor_area_m2,reference_wall_area_m2,reference_window_area_m2,reference_roof_area_m2
building_archetype,,,,,,,,,,,,,,
AT_ABL_1850_1918,AT_ABL_1850_1918,ZonWallIntB,9.574975,4.0,2016-01-01,2016-01-02,2.6,0.139171,0.5,0.0,512.800000,1130.380000,182.750000,512.800000
AT_ABL_1919_1944,AT_ABL_1919_1944,ZonWallIntB,9.404345,4.0,2016-01-01,2016-01-02,2.6,0.122862,0.5,0.0,493.000000,1128.000000,158.000000,493.000000
AT_ABL_1945_1960,AT_ABL_1945_1960,ZonWallIntB,15.004444,4.0,2016-01-01,2016-01-02,2.6,0.164607,0.5,0.0,337.700000,588.000000,115.860000,337.700000
AT_ABL_1961_1980,AT_ABL_1961_1980,ZonWallIntB,8.346839,4.0,2016-01-01,2016-01-02,2.6,0.162841,0.5,0.0,389.900000,958.740000,186.490000,389.900000
AT_ABL_1981_1990,AT_ABL_1981_1990,ZonWallIntB,8.228929,4.0,2016-01-01,2016-01-02,2.6,0.142344,0.5,0.0,425.340000,1068.880000,177.400000,425.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SK_TRA_1970_1979,SK_TRA_1970_1979,ZonWallIntB,71.632247,1.0,2016-01-01,2016-01-02,2.6,0.574935,0.5,0.0,25915.127956,957.988281,1295.756398,25915.127956
SK_TRA_1980_1989,SK_TRA_1980_1989,ZonWallIntB,71.632247,1.0,2016-01-01,2016-01-02,2.6,0.574935,0.5,0.0,25915.127956,957.988281,1295.756398,25915.127956
SK_TRA_1990_1999,SK_TRA_1990_1999,ZonWallIntB,71.632247,1.0,2016-01-01,2016-01-02,2.6,0.574935,0.5,0.0,25915.127956,957.988281,1295.756398,25915.127956


In [7]:
### Check if building frame depth values have `nan` due to complex sqrt

archetype_data[archetype_data["building_frame_depth_m"].isna()]

# Which should return an empty dataframe

,building_scope,building_fabrics,building_frame_depth_m,number_of_storeys,weather_start,weather_end,room_height_m,window_area_to_external_wall_ratio_m2_m2,partition_wall_length_ratio_to_external_walls_m_m,window_area_thermal_bridge_surcharge_W_m2K,reference_floor_area_m2,reference_wall_area_m2,reference_window_area_m2,reference_roof_area_m2
building_archetype,,,,,,,,,,,,,,


In [8]:
### Check mean values

cols = [
    "building_frame_depth_m",
    "number_of_storeys",
    "room_height_m",
    "window_area_to_external_wall_ratio_m2_m2",
    "reference_floor_area_m2",
    "reference_wall_area_m2",
    "reference_window_area_m2",
    "reference_roof_area_m2",
]

archetype_data[cols].mean()

building_frame_depth_m                        8.583670
number_of_storeys                             3.733459
room_height_m                                 2.600000
window_area_to_external_wall_ratio_m2_m2      0.196948
reference_floor_area_m2                     626.972684
reference_wall_area_m2                      650.694515
reference_window_area_m2                    276.572658
reference_roof_area_m2                      630.854231
dtype: float64

In [9]:
### Check median values

archetype_data[cols].median()

building_frame_depth_m                        7.026580
number_of_storeys                             3.000000
room_height_m                                 2.600000
window_area_to_external_wall_ratio_m2_m2      0.164997
reference_floor_area_m2                     220.000000
reference_wall_area_m2                      540.070248
reference_window_area_m2                    116.400000
reference_roof_area_m2                      226.000000
dtype: float64

On average, the parameters seem reasonable enough,
although average `building_frame_depth_m` less than 8-6 metres seems a bit low-ish from a Finnish point-of-view.

In [10]:
### Check maximum values

archetype_data[cols].max()

building_frame_depth_m                         71.632247
number_of_storeys                              27.000000
room_height_m                                   2.600000
window_area_to_external_wall_ratio_m2_m2        0.964454
reference_floor_area_m2                     25915.127956
reference_wall_area_m2                       9579.400000
reference_window_area_m2                    13672.800000
reference_roof_area_m2                      25915.127956
dtype: float64

The maximum values don't seem entirely unreasonable either,
with the possible exception of a 71 m maximum building frame depth.
Fortunately, it would seem that excessive building frame depths mostly occur
in non-residential buildings, and large warehouses could reasonably have shapes like that.

In [11]:
archetype_data[archetype_data["building_frame_depth_m"] >= 30]

,building_scope,building_fabrics,building_frame_depth_m,number_of_storeys,weather_start,weather_end,room_height_m,window_area_to_external_wall_ratio_m2_m2,partition_wall_length_ratio_to_external_walls_m_m,window_area_thermal_bridge_surcharge_W_m2K,reference_floor_area_m2,reference_wall_area_m2,reference_window_area_m2,reference_roof_area_m2
building_archetype,,,,,,,,,,,,,,
CZ_TRA_1980_1989,CZ_TRA_1980_1989,ZonWallIntB,34.290008,1.0,2016-01-01,2016-01-02,2.6,0.275219,0.5,0.0,5938.426377,781.935284,296.921319,5938.426377
EE_EDU_1850_1944,EE_EDU_1850_1944,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_1945_1969,EE_EDU_1945_1969,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_1970_1979,EE_EDU_1970_1979,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_1980_1989,EE_EDU_1980_1989,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_1990_1999,EE_EDU_1990_1999,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_2000_2010,EE_EDU_2000_2010,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
EE_EDU_2011_2021,EE_EDU_2011_2021,ZonWallIntB,39.566172,2.0,2016-01-01,2016-01-02,2.6,0.952698,0.5,0.0,7906.500000,117.768056,2371.950000,7906.500000
LV_HEA_1850_1944,LV_HEA_1850_1944,ZonWallIntB,39.388008,5.0,2016-01-01,2016-01-02,2.6,0.948408,0.5,0.0,7835.455436,319.675980,5876.591577,7835.455436


In [12]:
### Check minimum values

archetype_data[cols].min()

building_frame_depth_m                       0.256904
number_of_storeys                            1.000000
room_height_m                                2.600000
window_area_to_external_wall_ratio_m2_m2     0.000000
reference_floor_area_m2                      0.333333
reference_wall_area_m2                      24.098711
reference_window_area_m2                     0.000000
reference_roof_area_m2                       0.333333
dtype: float64

However, the minimum values get pretty unrealistic.
Cyprus has some clearly unreasonable data, with less than 1 m2 of floor area per storey. There are also several buildings with ground floor areas less than 10 m2,
which I also find quite suspicious.

In [13]:
ambience.data[ambience.data["REFERENCE BUILDING GROUND FLOOR AREA (m2)"] <= 10.0]

,building_type,building_period,location_id,REFERENCE BUILDING CONSTRUCTION YEAR LOW,REFERENCE BUILDING CONSTRUCTION YEAR HIGH,REFERENCE BUILDING MATERIAL COMBINATION CODE,average_gross_floor_area_m2_per_building,number_of_buildings,NUMBER OF REFERENCE BUILDING STOREYS,REFERENCE BUILDING GROUND FLOOR AREA (m2),...,HEATING SYSTEM 3 HEAT SOURCE,total_area_over_material_combinations_m2,material_combination_weight,shapefile_path,notes,category,raster_weight_path,notes_building_type,building_stock_year,building_stock
REFERENCE BUILDING CODE,,,,,,,,,,,,,,,,,,,,,
AT-HEA-1945-1969-00,HEA,1945-1969,AT,1945,1969,0,48.813667,65145.689431,5,9.762733,...,NaN,48.813667,1.0,../AmBIENCe2ABM/data_sources/eurostat/AT.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Healthcare buildings,2016,AmBIENCe_2016_AT_nonres
AT-HEA-1970-1979-00,HEA,1970-1979,AT,1970,1979,0,49.727468,21517.283124,5,9.945494,...,NaN,49.727468,1.0,../AmBIENCe2ABM/data_sources/eurostat/AT.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Healthcare buildings,2016,AmBIENCe_2016_AT_nonres
AT-HEA-1980-1989-00,HEA,1980-1989,AT,1980,1989,0,49.461728,22037.240594,5,9.892346,...,NaN,49.461728,1.0,../AmBIENCe2ABM/data_sources/eurostat/AT.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Healthcare buildings,2016,AmBIENCe_2016_AT_nonres
AT-HEA-2011-2021-00,HEA,2011-2021,AT,2011,2021,0,49.624794,6246.877310,5,9.924959,...,NaN,49.624794,1.0,../AmBIENCe2ABM/data_sources/eurostat/AT.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Healthcare buildings,2016,AmBIENCe_2016_AT_nonres
CY-OTH-1850-1944-01,OTH,1850-1944,CY,1850,1944,1,1.000000,49387.870977,3,0.333333,...,NaN,2.000000,0.5,../AmBIENCe2ABM/data_sources/eurostat/CY.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Other non-residential buildings,2016,AmBIENCe_2016_CY_nonres
CY-OTH-1945-1969-01,OTH,1945-1969,CY,1945,1969,1,1.000000,16158.971328,3,0.333333,...,NaN,2.000000,0.5,../AmBIENCe2ABM/data_sources/eurostat/CY.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Other non-residential buildings,2016,AmBIENCe_2016_CY_nonres
CY-OTH-1970-1979-01,OTH,1970-1979,CY,1970,1979,1,1.000000,10462.588531,3,0.333333,...,NaN,2.000000,0.5,../AmBIENCe2ABM/data_sources/eurostat/CY.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Other non-residential buildings,2016,AmBIENCe_2016_CY_nonres
CY-OTH-1980-1989-01,OTH,1980-1989,CY,1980,1989,1,1.000000,11222.106238,3,0.333333,...,NaN,2.000000,0.5,../AmBIENCe2ABM/data_sources/eurostat/CY.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Other non-residential buildings,2016,AmBIENCe_2016_CY_nonres
CY-OTH-1990-1999-01,OTH,1990-1999,CY,1990,1999,1,1.000000,12361.382797,3,0.333333,...,NaN,2.000000,0.5,../AmBIENCe2ABM/data_sources/eurostat/CY.shp,"AmBIENCe, Hotmaps, and NUTS available.",nonres,../AmBIENCe2ABM/data_sources/gfa_nonres_curr_d...,Other non-residential buildings,2016,AmBIENCe_2016_CY_nonres


## Forming `building_scope`

The `building_scope` objects define how the building stock statistics are aggregated
into the properties of the desired archetype buildings.
For replicating the reference buildings in the AmBIENCe data as close ly as possible,
the heat sources are re-aggregated, but the material combinations neglected.

In [14]:
### Check building scopes

defs.building_scope()

,scope_period_start_year,scope_period_end_year
building_scope,,
AT_OFF_1850_1944,1850,1944
AT_OFF_1945_1969,1945,1969
AT_OFF_1970_1979,1970,1979
AT_OFF_1980_1989,1980,1989
AT_OFF_1990_1999,1990,1999
...,...,...
SK_ABL_1921_1945,1921,1945
SK_ABL_1946_1960,1946,1960
SK_ABL_1961_1980,1961,1980


In [15]:
### Check building scope to building stock mapping

defs.building_scope__building_stock()

,building_stock
building_scope,
AT_OFF_1850_1944,AmBIENCe_2016_AT_nonres
AT_OFF_1945_1969,AmBIENCe_2016_AT_nonres
AT_OFF_1970_1979,AmBIENCe_2016_AT_nonres
AT_OFF_1980_1989,AmBIENCe_2016_AT_nonres
AT_OFF_1990_1999,AmBIENCe_2016_AT_nonres
...,...
SK_ABL_1921_1945,AmBIENCe_2016_SK_res
SK_ABL_1946_1960,AmBIENCe_2016_SK_res
SK_ABL_1961_1980,AmBIENCe_2016_SK_res


In [16]:
### Check building scope to building type mapping

defs.building_scope__building_type()

,building_type
building_scope,
AT_OFF_1850_1944,OFF
AT_OFF_1945_1969,OFF
AT_OFF_1970_1979,OFF
AT_OFF_1980_1989,OFF
AT_OFF_1990_1999,OFF
...,...
SK_ABL_1921_1945,ABL
SK_ABL_1946_1960,ABL
SK_ABL_1961_1980,ABL


In [17]:
### Check building scope to heat source mapping

defs.building_scope__heat_source()

,heat_source
building_scope,
AT_OFF_1850_1944,Electricity
AT_OFF_1850_1944,Gas
AT_OFF_1945_1969,Electricity
AT_OFF_1945_1969,Gas
AT_OFF_1970_1979,Electricity
...,...
SK_ABL_1921_1945,Electricity
SK_ABL_1946_1960,Electricity
SK_ABL_1961_1980,Electricity


In [18]:
### Check building scope to location ID mapping

defs.building_scope__location_id()

,location_id
building_scope,
AT_OFF_1850_1944,AT
AT_OFF_1945_1969,AT
AT_OFF_1970_1979,AT
AT_OFF_1980_1989,AT
AT_OFF_1990_1999,AT
...,...
SK_ABL_1921_1945,SK
SK_ABL_1946_1960,SK
SK_ABL_1961_1980,SK


## Test export and data packaging

Similar to the building stock data, the definitions can be packaged to make them
easier to deal with in Spine Toolbox.
This functionality is tested below.*

In [19]:
### Test data package export

defs.export_csvs()
pkg = defs.create_datapackage()
pkg

c:\Users\trtopi\AppData\Local\miniconda3\envs\atlitetoolbox\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


{'name': 'ambience2abm_definitions',
 'title': 'AmBIENCe2ABM reference building definitions',
 'description': 'A reference building definition package processed from '
                'AmBIENCe project EU27 data for use with '
                'ArchetypeBuildingModel.jl.',
 'homepage': 'https://github.com/spine-tools/AmBIENCe2ABM.jl',
 'profile': 'data-package',
 'licenses': [{'name': 'CC-BY-4.0',
               'path': 'https://creativecommons.org/licenses/by/4.0/',
               'title': 'Creative Commons Attribution 4.0'}],
 'sources': [{'name': 'D4.1 Database of grey-box model parameter values for EU '
                      'building typologies',
              'web': 'https://ambience-project.eu/wp-content/uploads/2022/02/AmBIENCe_D4.1_Database-of-grey-box-model-parameter-values-for-EU-building-typologies-update-version-2-submitted.pdf'},
             {'name': 'Database of grey-box model parameters',
              'web': 'https://ambience-project.eu/wp-content/uploads/2022/03/AmBIE